In [18]:
from tscsRL.environments.TSCSEnv import DiscreteTSCSEnv, ContinuousTSCSEnv
from tscsRL.environments.GradientTSCSEnv import DiscreteGradientTSCSEnv, ContinuousGradientTSCSEnv
from tscsRL.agents import ddqn
from tscsRL import utils
import imageio
import torch

In [19]:
name = 'ddqnGradient4cyl0.45-0.35-8000decay'
path = 'results/' + name
env_params = utils.jsonToDict(path + '/env_params.json')
agent_params = utils.jsonToDict(path + '/agent_params.json')

In [27]:
env = DiscreteGradientTSCSEnv(
    nCyl=env_params['nCyl'],
    kMax=env_params['kMax'],
    kMin=env_params['kMin'],
    nFreq=env_params['nFreq'],
    stepSize=env_params['stepSize'])

agent = ddqn.DDQNAgent(
    env.observation_space,
    env.action_space,
    agent_params,
    name)
agent.epsilon = 0.1

In [28]:
agent.load_checkpoint(path + '/checkpoints/', 8000)

In [38]:
## Starting from a random config
# state = env.reset()
## End starting from random config

## Starting from a predefined config
env.config = torch.tensor([[ 4.9073,  0.9211,  2.3434, -4.5528,  2.6180,  4.6988, -2.3804,  1.6361]])
env.counter = torch.tensor([[0.0]])
env.setMetric(env.config)

env.info['initial'] = env.RMS.item()
env.info['lowest'] = env.info['initial']
env.info['final'] = None
env.info['score'] = 0
state = env.getState()
## End starting from random config

done = False

results = {
    'config': [],
    'rms': [],
    'tscs': []}

while not done:
    results['config'].append(env.config)
    results['rms'].append(env.RMS)
    results['tscs'].append(env.TSCS)

    action = agent.select_action(state)
    nextState, reward, done, info = env.step(action)

#     print(reward, done)
    state = nextState

## Initial stuff
initialConfig = results['config'][0]
initialRMS = results['rms'][0]
initialTSCS = results['tscs'][0]

## Optimal stuff
minIdx = results['rms'].index(min(results['rms']))
optimalConfig = results['config'][minIdx]
optimalRMS = results['rms'][minIdx]
optimalTSCS = results['tscs'][minIdx]

print('RESULTS:')
print(f'Initial config: {initialConfig}')
print(f'Initial RMS: {initialRMS}')
print(f'Initial TSCS: {initialTSCS}')
print()
print(f'Min config: {optimalConfig}')
print(f'Min rms: {optimalRMS}')
print(f'Min tscs: {optimalTSCS}')

RESULTS:
Initial config: tensor([[ 4.9073,  0.9211,  2.3434, -4.5528,  2.6180,  4.6988, -2.3804,  1.6361]])
Initial RMS: tensor([[1.7649]])
Initial TSCS: tensor([[0.8088, 0.9470, 1.1028, 1.2736, 1.4560, 1.6464, 1.8408, 2.0357, 2.2278,
         2.4141, 2.5926]])

Min config: tensor([[ 0.4073,  3.9211,  0.3434, -3.0528, -2.8820,  3.6988, -3.3804, -2.8639]])
Min rms: tensor([[0.3910]])
Min tscs: tensor([[0.4567, 0.4184, 0.3837, 0.3551, 0.3350, 0.3257, 0.3290, 0.3466, 0.3800,
         0.4302, 0.4982]])
